# TM10007 Assignment template

In [ ]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [9]:
# Data loading functions. Uncomment the one you want to use
#from adni.load_data import load_data
#from brats.load_data import load_data
#from hn.load_data import load_data
from ecg.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
print(data)


The number of samples: 827
The number of columns: 9001
            0_0         0_1         0_2        0_3        0_4         0_5  \
0     47.685046   48.416904   37.371002  61.604757  58.732738  213.067966   
1    152.726718  297.353726  111.168880  25.419423  14.636085   24.609688   
2      1.601260    3.882169   22.978997  21.673040   8.919484   18.329286   
3      1.388947    3.052483    3.084103   4.627886  10.016196   15.020347   
4      3.625561    3.728466    6.205367  17.722897   5.699401    9.024836   
..          ...         ...         ...        ...        ...         ...   
822   14.673713   13.879148   10.000083   8.337630   3.557302    8.910661   
823    3.167367   13.771749    0.705951   0.349087   9.596407   14.721906   
824    0.439357    8.700374   20.628752  19.379063  23.176646   25.308323   
825    0.235515    4.074581    9.381945   9.621589  16.106126    8.376319   
826    0.301328    2.012000    9.122957   7.332689   7.862709   10.139178   

            0_6     

In [34]:
#print(data[data['label'] == 1])
y = data['label']
X = data.loc[:, data.columns != 'label']
# X = data.drop(labels = 'label')
print(X)

            0_0         0_1         0_2        0_3        0_4         0_5  \
0     47.685046   48.416904   37.371002  61.604757  58.732738  213.067966   
1    152.726718  297.353726  111.168880  25.419423  14.636085   24.609688   
2      1.601260    3.882169   22.978997  21.673040   8.919484   18.329286   
3      1.388947    3.052483    3.084103   4.627886  10.016196   15.020347   
4      3.625561    3.728466    6.205367  17.722897   5.699401    9.024836   
..          ...         ...         ...        ...        ...         ...   
822   14.673713   13.879148   10.000083   8.337630   3.557302    8.910661   
823    3.167367   13.771749    0.705951   0.349087   9.596407   14.721906   
824    0.439357    8.700374   20.628752  19.379063  23.176646   25.308323   
825    0.235515    4.074581    9.381945   9.621589  16.106126    8.376319   
826    0.301328    2.012000    9.122957   7.332689   7.862709   10.139178   

            0_6         0_7         0_8         0_9  ...    11_740    11_74

In [37]:
from sklearn.model_selection import StratifiedKFold
y = data['label']
X = data.loc[:, data.columns != 'label']
skf = StratifiedKFold(n_splits = 10, shuffle = True)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]



ImportError: DLL load failed while importing _arpack: Kan opgegeven procedure niet vinden.